In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import networkx as nx
from fastnode2vec import Graph, Node2Vec
from tqdm.notebook import tqdm

In [2]:
challange_data = pd.read_csv("../Data/challenge_data.csv")
train_data = pd.read_csv("../Data/train.csv")
test_data = pd.read_csv("../Data/test.csv")
sample_submission = pd.read_csv("../Data/sample_submission_J0OjXLi_DDt3uQN.csv")

In [3]:
train_data = train_data.merge(challange_data.rename(columns = {'challenge_ID' : 'challenge'}), on = 'challenge', how = 'left')
test_data = test_data.merge(challange_data.rename(columns = {'challenge_ID' : 'challenge'}), on = 'challenge', how = 'left')

In [4]:
transition_matrix = train_data[train_data.challenge_sequence <= 10][['user_id', 'challenge_sequence', 'challenge']].append(
                                            test_data[['user_id', 'challenge_sequence', 'challenge']]).sort_values(
                                            by = ['user_id', 'challenge_sequence'])[['user_id', 'challenge']]

transition_matrices = []
for i in range(1,10):
    transition_matrix1 = transition_matrix.copy()
    transition_matrix1['next_challenge'] = transition_matrix1[['user_id', 'challenge']].groupby('user_id').shift(-i)
    transition_matrix1 = transition_matrix1[transition_matrix1.next_challenge.notnull()]
    transition_matrix1['count'] = 1 / ((i - 1)//3 +1)
    transition_matrices.append(transition_matrix1.copy())
transition_matrix = pd.concat(transition_matrices)
# transition_matrix['next_challenge'] = transition_matrix[['user_id', 'challenge']].groupby('user_id').shift(-1)
# transition_matrix = transition_matrix[transition_matrix.next_challenge.notnull()]
# transition_matrix['next_challenge_skip_1'] = transition_matrix[['user_id', 'challenge']].groupby('user_id').shift(-2)
# transition_matrix['next_challenge_skip_2'] = transition_matrix[['user_id', 'challenge']].groupby('user_id').shift(-3)
# transition_matrix = transition_matrix[['challenge', 'next_challenge']].append(transition_matrix[
#                     transition_matrix.next_challenge_skip_1.notnull()][['challenge', 'next_challenge_skip_1']
#                     ].rename(columns = {'next_challenge_skip_1': 'next_challenge'})).append(transition_matrix[
#                     transition_matrix.next_challenge_skip_2.notnull()][['challenge', 'next_challenge_skip_2']
#                     ].rename(columns = {'next_challenge_skip_2': 'next_challenge'}))


transition_matrix = transition_matrix[['challenge', 'next_challenge', 'count']].groupby(
                                            ['challenge', 'next_challenge']).sum().reset_index()
transition_matrix['weight'] =  transition_matrix[['challenge', 'count']].groupby('challenge').transform('sum')
transition_matrix['weight'] = transition_matrix['count'] / transition_matrix['weight']
transition_matrix

,challenge,next_challenge,count,weight
0,CI23478,CI23491,0.5,0.030303
1,CI23478,CI23514,1.0,0.060606
2,CI23478,CI23606,1.0,0.060606
3,CI23478,CI23608,0.5,0.030303
4,CI23478,CI23653,1.0,0.060606
...,...,...,...,...
448721,CI29083,CI24917,1.0,0.222222
448722,CI29083,CI25409,1.0,0.222222
448723,CI29083,CI28570,1.0,0.222222
448724,CI29083,CI28698,0.5,0.111111


In [5]:

transition_matrix_series = train_data[train_data.challenge_sequence <= 10][['user_id', 'challenge_sequence', 'challenge_series_ID']].append(
                                            test_data[['user_id', 'challenge_sequence', 'challenge_series_ID']]).sort_values(
                                            by = ['user_id', 'challenge_sequence'])[['user_id', 'challenge_series_ID']]

transition_matrices = []
for i in range(1,10):
    transition_matrix1 = transition_matrix_series.copy()
    transition_matrix1['next_series'] = transition_matrix1[['user_id', 'challenge_series_ID']].groupby('user_id').shift(-i)
    transition_matrix1 = transition_matrix1[transition_matrix1.next_series.notnull()]
    transition_matrix1['count'] = 1 / ((i - 1)//3 +1)
    transition_matrices.append(transition_matrix1.copy())
transition_matrix_series = pd.concat(transition_matrices)

# transition_matrix_series['next_series'] = transition_matrix_series[['user_id', 'challenge_series_ID']].groupby('user_id').shift(-1)
# transition_matrix_series = transition_matrix_series[transition_matrix_series.next_series.notnull()]
# transition_matrix_series['next_series_skip_1'] = transition_matrix_series[['user_id', 'challenge_series_ID']].groupby('user_id').shift(-2)
# transition_matrix_series['next_series_skip_2'] = transition_matrix_series[['user_id', 'challenge_series_ID']].groupby('user_id').shift(-3)
# transition_matrix_series = transition_matrix_series[['challenge_series_ID', 'next_series']].append(transition_matrix_series[
#                     transition_matrix_series.next_series_skip_1.notnull()][['challenge_series_ID', 'next_series_skip_1']
#                     ].rename(columns = {'next_series_skip_1': 'next_series'})).append(transition_matrix_series[
#                     transition_matrix_series.next_series_skip_2.notnull()][['challenge_series_ID', 'next_series_skip_2']
#                     ].rename(columns = {'next_series_skip_2': 'next_series'}))

# transition_matrix_series['count'] = 1
transition_matrix_series = transition_matrix_series[['challenge_series_ID', 'next_series', 'count']].groupby(
                                            ['challenge_series_ID', 'next_series']).sum().reset_index()
transition_matrix_series['weight'] =  transition_matrix_series[['challenge_series_ID', 'count']].groupby('challenge_series_ID').transform('sum')
transition_matrix_series['weight'] = transition_matrix_series['count'] / transition_matrix_series['weight']
transition_matrix_series

,challenge_series_ID,next_series,count,weight
0,SI2435,SI2435,61.500000,0.025775
1,SI2435,SI2441,0.333333,0.000140
2,SI2435,SI2446,6.000000,0.002515
3,SI2435,SI2447,18.500000,0.007754
4,SI2435,SI2448,18.333333,0.007684
...,...,...,...,...
41176,SI2866,SI2675,1.000000,0.090909
41177,SI2866,SI2678,1.000000,0.090909
41178,SI2866,SI2739,1.000000,0.090909
41179,SI2866,SI2772,1.000000,0.090909


In [6]:
graph1 = Graph([(row.challenge, row.next_challenge, row.weight) for row in transition_matrix.itertuples()]
#                + [(str(row.user_id), row.challenge, 1) for row in 
#                 train_data[train_data.challenge_sequence <= 10].append(test_data).itertuples()]
               ,
              directed=True, weighted=True)

n2v1 = Node2Vec(graph1, dim=32, walk_length=20, context=10, p=2.0, q=0.5, workers=16)

n2v1.train(epochs=100)


Training: 100%|██████████| 523900/523900 [00:23<00:00, 22544.18it/s]


In [7]:
graph2 = Graph([(str(row.user_id), row.challenge) for row in 
                train_data[train_data.challenge_sequence <= 10].append(test_data).itertuples()] + [(row.challenge_series_ID, 
                row.challenge) for row in train_data[train_data.challenge_sequence <= 10
                ].append(test_data).itertuples()] + [(row.author_org_ID, 
                row.challenge) for row in train_data[train_data.challenge_sequence <= 10
                ].append(test_data).itertuples()] + [(row.author_ID, 
                row.challenge) for row in train_data[train_data.challenge_sequence <= 10
                ].append(test_data).itertuples()],
              directed=False, weighted=False)

n2v2 = Node2Vec(graph2, dim=32, walk_length=20, context=10, p=2.0, q=0.5, workers=16)

n2v2.train(epochs=100)


Training: 100%|██████████| 11981100/11981100 [10:22<00:00, 19259.78it/s]


In [8]:
train_data.head()

,user_sequence,user_id,challenge_sequence,challenge,programming_language,challenge_series_ID,total_submissions,publish_date,author_ID,author_gender,author_org_ID,category_id
0,4576_1,4576,1,CI23714,1,SI2477,14723.0,12-06-2006,AI563766,M,AOI100129,29.0
1,4576_2,4576,2,CI23855,1,SI2468,20993.0,24-07-2006,AI563823,M,NaN,29.0
2,4576_3,4576,3,CI24917,1,SI2545,43409.0,02-07-2007,AI564588,F,AOI100581,66.0
3,4576_4,4576,4,CI23663,1,SI2472,8897.0,15-09-2004,AI563724,M,AOI100098,45.0
4,4576_5,4576,5,CI23933,1,SI2468,15086.0,27-07-2006,AI563783,M,AOI100149,31.0


In [9]:
train_users = train_data.user_id.unique()
already_done_challenges = train_data[train_data.challenge_sequence <=10][['user_id', 'challenge']
                                                                        ].groupby('user_id').agg(set).challenge.to_dict()
n2v_similarities = {}
def get_n2v_similarity(a,b):
    try:
        return n2v_similarities[str(a), str(b)]
    except KeyError:
        try:
            res = n2v2.similarity(str(a), str(b))
            n2v_similarities[str(a), str(b)] = res
            return res
        except KeyError:
            n2v_similarities[str(a), str(b)] = -1
            return -1

train = []
for user_id in tqdm(train_data.user_id.unique()):
    challenges = set([similar_challenge for challenge in already_done_challenges[user_id] 
                    for similar_challenge, score in n2v1.most_similar(challenge, topn = 50) if similar_challenge[0] == 'C'])
    challenges = list(challenges - already_done_challenges[user_id])
    train.append(pd.DataFrame({'user_id' : [user_id]*len(challenges), 
                              "challenge" : challenges, 
                               'n2v_similarity' : [get_n2v_similarity(user_id, challenge) for challenge in challenges]}))
    
train = pd.concat(train)


test_users = test_data.user_id.unique()
already_done_challenges = test_data[test_data.challenge_sequence <=10][['user_id', 'challenge']
                                                                        ].groupby('user_id').agg(set).challenge.to_dict()

test = []
for user_id in tqdm(test_data.user_id.unique()):
    challenges = set([similar_challenge for challenge in already_done_challenges[user_id] 
                    for similar_challenge, score in n2v1.most_similar(challenge, topn = 50) if similar_challenge[0] == 'C'])
    challenges = list(challenges - already_done_challenges[user_id])
    test.append(pd.DataFrame({'user_id' : [user_id]*len(challenges), 
                              "challenge" : challenges, 
                               'n2v_similarity' : [get_n2v_similarity(user_id, challenge) for challenge in challenges]}))
test = pd.concat(test)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # Remove the CWD from sys.path while we load stuff.


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [10]:
train

,user_id,challenge,n2v_similarity
0,4576,CI23798,0.315841
1,4576,CI24968,0.643808
2,4576,CI23781,0.626791
3,4576,CI26050,0.669567
4,4576,CI23691,0.689923
...,...,...,...
313,113839,CI26579,0.400110
314,113839,CI24228,0.463715
315,113839,CI28897,0.484968
316,113839,CI24252,0.564924


In [11]:
train.shape

(11530183, 3)

In [12]:
user_challenges = train_data[['user_id', 'challenge_sequence', 'challenge']].pivot(
                                    index = 'user_id', values = 'challenge', columns = 'challenge_sequence').reset_index()
user_challenges.columns = ['user_id'] + ["challenge_" + str(c) for c in user_challenges.columns[1:]]
train = train.merge(user_challenges, on = 'user_id', how = 'left')

user_challenges = test_data[['user_id', 'challenge_sequence', 'challenge']].pivot(
                                    index = 'user_id', values = 'challenge', columns = 'challenge_sequence').reset_index()
user_challenges.columns = ['user_id'] + ["challenge_" + str(c) for c in user_challenges.columns[1:]]
test = test.merge(user_challenges, on = 'user_id', how = 'left')

In [13]:
(train.challenge == train.challenge_11).sum(), (train.challenge == train.challenge_12).sum(), (train.challenge == train.challenge_13).sum()

(56773, 54566, 52809)

In [14]:
train['target'] = ((train.challenge == train.challenge_11).astype(int) + (train.challenge == train.challenge_12).astype(int) 
                   + (train.challenge == train.challenge_13).astype(int)).map(lambda x : 1 if x > 0 else 0)

In [15]:
## theoretical maximum
df = train.sort_values(by = 'target', ascending = False).groupby('user_id').head(3)
df['rank'] = df.groupby('user_id').target.rank(ascending = False, method = 'first')
# val1 = val1.sort_values(by = ['user_id', 'pred'], ascending = False)

df['precision'] = ((df['rank'] == 1) & (df.target == 1)) | ((df['rank'] ==  2) & 
                                    (df.target == 1)) | ((df['rank'] == 3) & (df.target == 1))
df['precision'] = df[['user_id', 'precision']].groupby('user_id').cumsum()
df['precision'] = df['precision'] / df['rank']
df[['user_id', 'precision']].groupby('user_id').mean().precision.mean()

0.8763494985523548

In [16]:
train1 = train.merge(challange_data.rename(columns = {'challenge_ID' : 'challenge'}), on = 'challenge', how = 'left')
test1 = test.merge(challange_data.rename(columns = {'challenge_ID' : 'challenge'}), on = 'challenge', how = 'left')

In [17]:
user_series_count = train_data[train_data.challenge_sequence <= 10][['user_id', 'challenge_series_ID', 'user_sequence']
                                            ].groupby(['user_id', 'challenge_series_ID']).count().user_sequence.to_dict()
challenge_series_mapping = challange_data[['challenge_ID', 'challenge_series_ID']
                                         ].set_index('challenge_ID').challenge_series_ID.to_dict()
train1['user_series_count'] = [user_series_count.get((row.user_id, challenge_series_mapping[row.challenge]), 0) 
                              for row in tqdm(train1.itertuples())]

user_series_count = test_data[test_data.challenge_sequence <= 10][['user_id', 'challenge_series_ID', 'user_sequence']
                                            ].groupby(['user_id', 'challenge_series_ID']).count().user_sequence.to_dict()
test1['user_series_count'] = [user_series_count.get((row.user_id, challenge_series_mapping[row.challenge]), 0) 
                              for row in tqdm(test1.itertuples())]

In [18]:
train1[train1.user_series_count > 0].target.mean(), train1.target.mean()

(0.0350509207540831, 0.01423637421886539)

In [19]:
user_num_unique_series = train_data[train_data.challenge_sequence <= 10][['user_id', 'challenge_series_ID']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
train1['user_num_unique_series'] = train1['user_id'].map(user_num_unique_series)

user_num_unique_series = test_data[test_data.challenge_sequence <= 10][['user_id', 'challenge_series_ID']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
test1['user_num_unique_series'] = test1['user_id'].map(user_num_unique_series)


In [20]:
train1[['user_num_unique_series', 'target']].groupby('user_num_unique_series').mean()

,target
user_num_unique_series,
1,0.046149
2,0.031373
3,0.018953
4,0.013491
5,0.011804
6,0.011162
7,0.010491
8,0.009519
9,0.008334


In [21]:
for key in ['challenge', 'challenge_series_ID', 'author_ID', 'author_org_ID', 'category_id']:
    key_user_counts = train_data[train_data.challenge_sequence <= 10][['user_id', key]].append(
                test_data[['user_id', key]]).drop_duplicates()[key].value_counts().to_dict()
    train1[key + "_user_count"] = train1[key].map(key_user_counts)
    test1[key + "_user_count"] = test1[key].map(key_user_counts)
    
    key_avg_seq = train_data[train_data.challenge_sequence <= 10][['challenge_sequence', key]].append(
                test_data[['challenge_sequence', key]]).groupby(key).mean().challenge_sequence.to_dict()
    train1[key + "_avg_seq"] = train1[key].map(key_avg_seq)
    test1[key + "_avg_seq"] = test1[key].map(key_avg_seq)
    
    

In [22]:
user_author_count = train_data[train_data.challenge_sequence <= 10][['user_id', 'author_ID', 'user_sequence']
                                            ].groupby(['user_id', 'author_ID']).count().user_sequence.to_dict()
challenge_author_mapping = challange_data[['challenge_ID', 'author_ID']
                                         ].set_index('challenge_ID').author_ID.to_dict()
train1['user_author_count'] = [user_author_count.get((row.user_id, challenge_author_mapping[row.challenge]), 0) 
                              for row in tqdm(train1.itertuples())]

user_author_count = test_data[test_data.challenge_sequence <= 10][['user_id', 'author_ID', 'user_sequence']
                                            ].groupby(['user_id', 'author_ID']).count().user_sequence.to_dict()
test1['user_author_count'] = [user_author_count.get((row.user_id, challenge_author_mapping[row.challenge]), 0) 
                              for row in tqdm(test1.itertuples())]


In [23]:
train1[train1.user_author_count > 0].target.mean(), train1.target.mean()

(0.042969782135390745, 0.01423637421886539)

In [24]:
user_num_unique_author = train_data[train_data.challenge_sequence <= 10][['user_id', 'author_ID']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
train1['user_num_unique_author'] = train1['user_id'].map(user_num_unique_author)

user_num_unique_author = test_data[test_data.challenge_sequence <= 10][['user_id', 'author_ID']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
test1['user_num_unique_author'] = test1['user_id'].map(user_num_unique_author)


In [25]:
train1[['user_num_unique_author', 'target']].groupby('user_num_unique_author').mean()

,target
user_num_unique_author,
1,0.041161
2,0.031859
3,0.028990
4,0.018389
5,0.011877
6,0.011239
7,0.010732
8,0.010346
9,0.010384


In [26]:
user_author_count = train_data[train_data.challenge_sequence <= 10][['user_id', 'author_org_ID', 'user_sequence']
                                            ].groupby(['user_id', 'author_org_ID']).count().user_sequence.to_dict()
challenge_author_mapping = challange_data[['challenge_ID', 'author_org_ID']
                                         ].set_index('challenge_ID').author_org_ID.to_dict()
train1['user_author_org_count'] = [user_author_count.get((row.user_id, challenge_author_mapping[row.challenge]), 0) 
                              for row in tqdm(train1.itertuples())]

user_author_count = test_data[test_data.challenge_sequence <= 10][['user_id', 'author_org_ID', 'user_sequence']
                                            ].groupby(['user_id', 'author_org_ID']).count().user_sequence.to_dict()
test1['user_author_org_count'] = [user_author_count.get((row.user_id, challenge_author_mapping[row.challenge]), 0) 
                              for row in tqdm(test1.itertuples())]


In [27]:
train1[train1.user_author_org_count > 0].target.mean(), train1.target.mean()

(0.03386625906420463, 0.01423637421886539)

In [28]:
user_num_unique_author_org = train_data[train_data.challenge_sequence <= 10][['user_id', 'author_org_ID']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
train1['user_num_unique_author_org'] = train1['user_id'].map(user_num_unique_author_org)

user_num_unique_author_org = test_data[test_data.challenge_sequence <= 10][['user_id', 'author_org_ID']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
test1['user_num_unique_author_org'] = test1['user_id'].map(user_num_unique_author_org)


In [29]:
train1[['user_num_unique_author_org', 'target']].groupby('user_num_unique_author_org').mean()

,target
user_num_unique_author_org,
1,0.040175
2,0.032137
3,0.018174
4,0.013594
5,0.011488
6,0.011183
7,0.010938
8,0.010659
9,0.010612


In [30]:
user_category_count = train_data[train_data.challenge_sequence <= 10][['user_id', 'category_id', 'user_sequence']
                                            ].groupby(['user_id', 'category_id']).count().user_sequence.to_dict()
challenge_category_mapping = challange_data[['challenge_ID', 'category_id']
                                         ].set_index('challenge_ID').category_id.to_dict()
train1['user_category_count'] = [user_category_count.get((row.user_id, challenge_category_mapping[row.challenge]), 0) 
                              for row in tqdm(train1.itertuples())]

user_category_count = test_data[test_data.challenge_sequence <= 10][['user_id', 'category_id', 'user_sequence']
                                            ].groupby(['user_id', 'category_id']).count().user_sequence.to_dict()
test1['user_category_count'] = [user_category_count.get((row.user_id, challenge_category_mapping[row.challenge]), 0) 
                              for row in tqdm(test1.itertuples())]


In [31]:
user_num_unique_category = train_data[train_data.challenge_sequence <= 10][['user_id', 'category_id']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
train1['user_num_unique_category'] = train1['user_id'].map(user_num_unique_category)

user_num_unique_category = test_data[test_data.challenge_sequence <= 10][['user_id', 'category_id']
                                            ].drop_duplicates().user_id.value_counts().to_dict()
test1['user_num_unique_category'] = test1['user_id'].map(user_num_unique_category)


In [32]:
train1[['user_num_unique_category', 'target']].groupby('user_num_unique_category').mean()

,target
user_num_unique_category,
1,0.041523
2,0.023733
3,0.015424
4,0.011646
5,0.010414
6,0.010077
7,0.009971
8,0.009920
9,0.009763


In [33]:
user_min_difficulty = train_data[train_data.challenge_sequence <= 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').max().total_submissions.to_dict()
user_max_difficulty = train_data[train_data.challenge_sequence <= 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').min().total_submissions.to_dict()
user_mean_difficulty = train_data[train_data.challenge_sequence <= 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').mean().total_submissions.to_dict()
user_last_difficulty = train_data[train_data.challenge_sequence == 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').mean().total_submissions.to_dict()

train1['user_min_difficulty'] = train1.user_id.map(user_min_difficulty)
train1['user_max_difficulty'] = train1.user_id.map(user_max_difficulty)
train1['user_mean_difficulty'] = train1.user_id.map(user_mean_difficulty)
train1['user_last_difficulty'] = train1.user_id.map(user_last_difficulty)

train1['user_min_difficulty_diff'] = train1['user_min_difficulty'] - train1.total_submissions
train1['user_max_difficulty_diff'] = train1['user_max_difficulty'] - train1.total_submissions
train1['user_mean_difficulty_diff'] = train1['user_mean_difficulty'] - train1.total_submissions
train1['user_last_difficulty_diff'] = train1['user_last_difficulty'] - train1.total_submissions



user_min_difficulty = test_data[test_data.challenge_sequence <= 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').max().total_submissions.to_dict()
user_max_difficulty = test_data[test_data.challenge_sequence <= 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').min().total_submissions.to_dict()
user_mean_difficulty = test_data[test_data.challenge_sequence <= 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').mean().total_submissions.to_dict()
user_last_difficulty = test_data[test_data.challenge_sequence == 10][['user_id', 'total_submissions']
                                                                ].groupby('user_id').mean().total_submissions.to_dict()

test1['user_min_difficulty'] = test1.user_id.map(user_min_difficulty)
test1['user_max_difficulty'] = test1.user_id.map(user_max_difficulty)
test1['user_mean_difficulty'] = test1.user_id.map(user_mean_difficulty)
test1['user_last_difficulty'] = test1.user_id.map(user_last_difficulty)

test1['user_min_difficulty_diff'] = test1['user_min_difficulty'] - test1.total_submissions
test1['user_max_difficulty_diff'] = test1['user_max_difficulty'] - test1.total_submissions
test1['user_mean_difficulty_diff'] = test1['user_mean_difficulty'] - test1.total_submissions
test1['user_last_difficulty_diff'] = test1['user_last_difficulty'] - test1.total_submissions


In [34]:
train_data.publish_date = pd.to_datetime(train_data.publish_date)
test_data.publish_date = pd.to_datetime(test_data.publish_date)

In [35]:
user_min_publish_date = train_data[train_data.challenge_sequence <= 10][['user_id', 'publish_date']
                                                                ].groupby('user_id').max().publish_date.to_dict()
user_max_publish_date = train_data[train_data.challenge_sequence <= 10][['user_id', 'publish_date']
                                                                ].groupby('user_id').min().publish_date.to_dict()
user_last_publish_date = train_data[train_data.challenge_sequence == 10][['user_id', 'publish_date']
                                                                ].groupby('user_id').last().publish_date.to_dict()

train1['user_min_publish_date'] = train1.user_id.map(user_min_publish_date)
train1['user_max_publish_date'] = train1.user_id.map(user_max_publish_date)
train1['user_last_publish_date'] = train1.user_id.map(user_last_publish_date)

train1.publish_date = pd.to_datetime(train1.publish_date)
train1['user_min_publish_date_diff'] = (train1['user_min_publish_date'] - train1.publish_date).dt.days
train1['user_max_publish_date_diff'] = (train1['user_max_publish_date'] - train1.publish_date).dt.days
train1['user_last_publish_date_diff'] = (train1['user_last_publish_date'] - train1.publish_date).dt.days



user_min_publish_date = test_data[test_data.challenge_sequence <= 10][['user_id', 'publish_date']
                                                                ].groupby('user_id').max().publish_date.to_dict()
user_max_publish_date = test_data[test_data.challenge_sequence <= 10][['user_id', 'publish_date']
                                                                ].groupby('user_id').min().publish_date.to_dict()
user_last_publish_date = test_data[test_data.challenge_sequence == 10][['user_id', 'publish_date']
                                                                ].groupby('user_id').last().publish_date.to_dict()

test1['user_min_publish_date'] = test1.user_id.map(user_min_publish_date)
test1['user_max_publish_date'] = test1.user_id.map(user_max_publish_date)
test1['user_last_publish_date'] = test1.user_id.map(user_last_publish_date)

test1.publish_date = pd.to_datetime(test1.publish_date)
test1['user_min_publish_date_diff'] = (test1['user_min_publish_date'] - test1.publish_date).dt.days
test1['user_max_publish_date_diff'] = (test1['user_max_publish_date'] - test1.publish_date).dt.days
test1['user_last_publish_date_diff'] = (test1['user_last_publish_date'] - test1.publish_date).dt.days


In [36]:
transition_scores = transition_matrix.set_index(['challenge', 'next_challenge']).weight.to_dict()
for i in range(1,11):
    col = "challenge_" + str(i)
    train1['transition_scores_' + str(i)] = [transition_scores.get((row[1], row[2]),0) 
                               for row in tqdm(train1[[col, 'challenge']].itertuples())]
    test1['transition_scores_' + str(i)] = [transition_scores.get((row[1], row[2]),0) 
                              for row in tqdm(test1[[col, 'challenge']].itertuples())]


In [37]:
transition_matrix_series.columns

Index(['challenge_series_ID', 'next_series', 'count', 'weight'], dtype='object')

In [38]:
transition_scores_series = transition_matrix_series.set_index(['challenge_series_ID', 'next_series']).weight.to_dict()

challenge_series_mapping = challange_data[['challenge_ID', 'challenge_series_ID']
                                         ].set_index('challenge_ID').challenge_series_ID.to_dict()


for i in range(1,11):
    col = "challenge_" + str(i)

    train1['transition_score_series_' + str(i)] = [transition_scores_series.get((challenge_series_mapping[row[1]], row[2]),0) 
                                   for row in tqdm(train1[['challenge_' + str(i), 'challenge_series_ID']].itertuples())]

    test1['transition_score_series_' + str(i)] = [transition_scores_series.get((challenge_series_mapping[row[1]], row[2]),0) 
                                  for row in tqdm(test1[['challenge_' + str(i), 'challenge_series_ID']].itertuples())]


In [39]:
train1[train1.transition_score_series_10> 0.1].target.mean(), train1.target.mean()

(0.06614255755935775, 0.01423637421886539)

In [40]:
train1['author_gender_cd'] = train1.author_gender.map(lambda x : 0 if x == 'F' else 1 if x == "M" else -1)
test1['author_gender_cd'] = test1.author_gender.map(lambda x : 0 if x == 'F' else 1 if x == "M" else -1)

In [41]:
train1['challenge_id_diff'] = train1.challenge.map(lambda x : int(x[2:])) - train1.challenge_10.map(lambda x : int(x[2:]))
df = train_data[train_data.challenge_sequence <= 10]
df['next_challenge'] = df[['user_id', 'challenge']].groupby('user_id').shift(-1)
df = df[df.next_challenge.notnull()]
df['challenge_id_diff'] = df.next_challenge.map(lambda x : int(x[2:])) - df.challenge.map(lambda x : int(x[2:]))
challenge_id_diff_mean = df[['user_id', 'challenge_id_diff']].groupby('user_id').mean().challenge_id_diff.to_dict()
train1['challenge_id_diff_mean'] = train1['user_id'].map(challenge_id_diff_mean)



test1['challenge_id_diff'] = test1.challenge.map(lambda x : int(x[2:])) - test1.challenge_10.map(lambda x : int(x[2:]))
df = test_data[test_data.challenge_sequence <= 10]
df['next_challenge'] = df[['user_id', 'challenge']].groupby('user_id').shift(-1)
df = df[df.next_challenge.notnull()]
df['challenge_id_diff'] = df.next_challenge.map(lambda x : int(x[2:])) - df.challenge.map(lambda x : int(x[2:]))
challenge_id_diff_mean = df[['user_id', 'challenge_id_diff']].groupby('user_id').mean().challenge_id_diff.to_dict()
test1['challenge_id_diff_mean'] = test1['user_id'].map(challenge_id_diff_mean)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
n2v_similarities = {}
def get_n2v_similarity(a,b):
    try:
        return n2v_similarities[str(a), str(b)]
    except KeyError:
        try:
            res = n2v1.similarity(str(a), str(b))
            n2v_similarities[str(a), str(b)] = res
            return res
        except KeyError:
            n2v_similarities[str(a), str(b)] = -1
            return -1
for i in range(1,11):
    train1['n2v1_challenge_similarity_' + str(i)] = [get_n2v_similarity(row[1], row[2]) for row in tqdm(train1[['challenge', 'challenge_' + str(i)]].itertuples())]
    test1['n2v1_challenge_similarity_' + str(i)] = [get_n2v_similarity(row[1], row[2]) for row in tqdm(test1[['challenge', 'challenge_' + str(i)]].itertuples())]


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


In [43]:
train1[train1.n2v1_challenge_similarity_10 > 0.9].target.mean(), train1.target.mean()

(0.04703797049460975, 0.01423637421886539)

In [44]:
n2v_similarities = {}
def get_n2v_similarity(a,b):
    try:
        return n2v_similarities[str(a), str(b)]
    except KeyError:
        try:
            res = n2v2.similarity(str(a), str(b))
            n2v_similarities[str(a), str(b)] = res
            return res
        except KeyError:
            n2v_similarities[str(a), str(b)] = -1
            return -1
for i in range(1,11):
    train1['n2v2_challenge_similarity_' + str(i)] = [get_n2v_similarity(row[1], row[2]) for row in tqdm(train1[['challenge', 'challenge_' + str(i)]].itertuples())]
    test1['n2v2_challenge_similarity_' + str(i)] = [get_n2v_similarity(row[1], row[2]) for row in tqdm(test1[['challenge', 'challenge_' + str(i)]].itertuples())]


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


In [45]:
n2v_similarities = {}
def get_n2v_similarity(a,b):
    try:
        return n2v_similarities[str(a), str(b)]
    except KeyError:
        try:
            res = n2v2.similarity(str(a), str(b))
            n2v_similarities[str(a), str(b)] = res
            return res
        except KeyError:
            n2v_similarities[str(a), str(b)] = -1
            return -1
    
    
challenge_author_mapping = challange_data[['challenge_ID', 'author_ID']
                                         ].set_index('challenge_ID').author_ID.to_dict()
for i in range(1,11):
    train1['n2v2_author_similarity_' + str(i)] = [get_n2v_similarity(row[1], challenge_author_mapping[row[2]]) 
                                              for row in tqdm(train1[['author_ID', "challenge_" + str(i)]].itertuples())]
    test1['n2v2_author_similarity_' + str(i)] = [get_n2v_similarity(row[1], challenge_author_mapping[row[2]]) 
                                             for row in tqdm(test1[['author_ID', "challenge_" + str(i)]].itertuples())]


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


In [46]:
train1[train1.n2v2_author_similarity_10 > 0.9].target.mean(), train1.target.mean()

(0.05910808680441991, 0.01423637421886539)

In [47]:
n2v_similarities = {}
def get_n2v_similarity(a,b):
    try:
        return n2v_similarities[str(a), str(b)]
    except KeyError:
        try:
            res = n2v2.similarity(str(a), str(b))
            n2v_similarities[str(a), str(b)] = res
            return res
        except KeyError:
            n2v_similarities[str(a), str(b)] = -1
            return -1
    
train1['n2v2_user_author_similarity'] = [get_n2v_similarity(row.user_id, row.author_ID) 
                                          for row in tqdm(train1.itertuples())]
test1['n2v2_user_author_similarity'] = [get_n2v_similarity(row.user_id, row.author_ID)
                                         for row in tqdm(test1.itertuples())]

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


In [48]:
train1[train1.n2v2_user_author_similarity > 0.9].target.mean(), train1.target.mean()

(0.08667545278887949, 0.01423637421886539)

In [49]:
for i in range(1,11):
    train1['challenge_id_diff_' + str(i)] = train1.challenge.str[2:].astype(int) - train1['challenge_' + str(i)].str[2:].astype(int)
    test1['challenge_id_diff_' + str(i)] = test1.challenge.str[2:].astype(int) - test1['challenge_' + str(i)].str[2:].astype(int)

In [50]:
train1[train1.challenge_id_diff_10.abs() < 10].target.mean(), train1.target.mean()

(0.11597704620960435, 0.01423637421886539)

In [51]:
# challenge_mean_sequence = train_data[train_data.challenge_sequence <= 10][['challenge', 'challenge_sequence']].append(
#                     test_data[['challenge',  'challenge_sequence']]).groupby('challenge').mean().challenge_sequence.to_dict()
# train1['challenge_mean_sequence'] = train1.challenge.map(challenge_mean_sequence)
# test1['challenge_mean_sequence'] = test1.challenge.map(challenge_mean_sequence)

In [53]:
print(list(train1.columns))

['user_id', 'challenge', 'n2v_similarity', 'challenge_1', 'challenge_2', 'challenge_3', 'challenge_4', 'challenge_5', 'challenge_6', 'challenge_7', 'challenge_8', 'challenge_9', 'challenge_10', 'challenge_11', 'challenge_12', 'challenge_13', 'target', 'programming_language', 'challenge_series_ID', 'total_submissions', 'publish_date', 'author_ID', 'author_gender', 'author_org_ID', 'category_id', 'user_series_count', 'user_num_unique_series', 'challenge_user_count', 'challenge_avg_seq', 'challenge_series_ID_user_count', 'challenge_series_ID_avg_seq', 'author_ID_user_count', 'author_ID_avg_seq', 'author_org_ID_user_count', 'author_org_ID_avg_seq', 'category_id_user_count', 'category_id_avg_seq', 'user_author_count', 'user_num_unique_author', 'user_author_org_count', 'user_num_unique_author_org', 'user_category_count', 'user_num_unique_category', 'user_min_difficulty', 'user_max_difficulty', 'user_mean_difficulty', 'user_last_difficulty', 'user_min_difficulty_diff', 'user_max_difficulty_di

In [54]:
indep_vars = ['n2v_similarity', 'programming_language', 'total_submissions', 'category_id', 'user_series_count', 
              'user_num_unique_series', 'challenge_user_count', 'challenge_avg_seq', 'challenge_series_ID_user_count', 
              'challenge_series_ID_avg_seq', 'author_ID_user_count', 'author_ID_avg_seq', 'author_org_ID_user_count', 
              'author_org_ID_avg_seq', 'category_id_user_count', 'category_id_avg_seq', 'user_author_count', 
              'user_num_unique_author', 'user_author_org_count', 'user_num_unique_author_org', 'user_category_count', 
              'user_num_unique_category', 'user_min_difficulty', 'user_max_difficulty', 'user_mean_difficulty', 
              'user_last_difficulty', 'user_min_difficulty_diff', 'user_max_difficulty_diff', 'user_mean_difficulty_diff',
              'user_last_difficulty_diff', 
              'user_min_publish_date_diff', 'user_max_publish_date_diff', 'user_last_publish_date_diff', 
              'transition_scores_1', 'transition_scores_2', 'transition_scores_3', 'transition_scores_4', 
              'transition_scores_5', 'transition_scores_6', 'transition_scores_7', 'transition_scores_8', 
              'transition_scores_9', 'transition_scores_10', 'transition_score_series_1', 'transition_score_series_2',
              'transition_score_series_3', 'transition_score_series_4', 'transition_score_series_5',
              'transition_score_series_6', 'transition_score_series_7', 'transition_score_series_8', 
              'transition_score_series_9', 'transition_score_series_10', 'author_gender_cd', 'challenge_id_diff',
              'challenge_id_diff_mean', 'n2v1_challenge_similarity_1', 'n2v1_challenge_similarity_2',
              'n2v1_challenge_similarity_3', 'n2v1_challenge_similarity_4', 'n2v1_challenge_similarity_5', 
              'n2v1_challenge_similarity_6', 'n2v1_challenge_similarity_7', 'n2v1_challenge_similarity_8',
              'n2v1_challenge_similarity_9', 'n2v1_challenge_similarity_10', 'n2v2_challenge_similarity_1',
              'n2v2_challenge_similarity_2', 'n2v2_challenge_similarity_3', 'n2v2_challenge_similarity_4',
              'n2v2_challenge_similarity_5', 'n2v2_challenge_similarity_6', 'n2v2_challenge_similarity_7',
              'n2v2_challenge_similarity_8', 'n2v2_challenge_similarity_9', 'n2v2_challenge_similarity_10', 
              'n2v2_author_similarity_1', 'n2v2_author_similarity_2', 'n2v2_author_similarity_3', 'n2v2_author_similarity_4',
              'n2v2_author_similarity_5', 'n2v2_author_similarity_6', 'n2v2_author_similarity_7', 'n2v2_author_similarity_8',
              'n2v2_author_similarity_9', 'n2v2_author_similarity_10', 'n2v2_user_author_similarity', 'challenge_id_diff_1', 
              'challenge_id_diff_2', 'challenge_id_diff_3', 'challenge_id_diff_4', 'challenge_id_diff_5',
              'challenge_id_diff_6', 'challenge_id_diff_7', 'challenge_id_diff_8', 'challenge_id_diff_9', 
              'challenge_id_diff_10', 
#               'challenge_mean_sequence'
             ]
len(indep_vars)

97

In [59]:
import gc
for var in ['user_min_publish_date', 'user_max_publish_date', 'user_last_publish_date', 'challenge_1', 'challenge_2', 
            'challenge_3', 'challenge_4', 'challenge_5', 'challenge_6', 'challenge_7', 'challenge_8', 'challenge_9',
            'challenge_10']:    
    try:
        del train1[var]
        del test1[var]
    except KeyError:
        pass
gc.collect()

20

In [60]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective':'binary',
    'metric': {'auc'},
    'num_leaves': 1800,
    'learning_rate': 0.025,
    'feature_fraction': 0.65,
    'bagging_fraction': 0.65,
    'bagging_freq': 1,
    'verbose': 1,
    'min_data_in_leaf' : 5,
    'max_bin' : 255,
    'lambda_l1' : 0.00002,
    'lambda_l2' : 0.00001,
    'min_gain_to_split' : 0.001
}
import random
random.seed(2)
dev_users = random.sample(list(set(train1.user_id)), 55000)
dev = train1[train1.user_id.isin(dev_users)]
val = train1[train1.user_id.isin(dev_users) == False]

print(dev.shape, val.shape)
lgb_dev = lgb.Dataset(dev[indep_vars], dev['target'])
lgb_val = lgb.Dataset(val[indep_vars], val['target'])

model = lgb.train(params, lgb_dev, num_boost_round = 3000, valid_sets = (lgb_val), early_stopping_rounds = 500,
                     verbose_eval = 100)

(9108181, 108) (2422002, 108)
Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.941025
[200]	valid_0's auc: 0.944859
[300]	valid_0's auc: 0.946522
[400]	valid_0's auc: 0.946453
[500]	valid_0's auc: 0.946425
[600]	valid_0's auc: 0.947008
[700]	valid_0's auc: 0.947466
[800]	valid_0's auc: 0.947976
[900]	valid_0's auc: 0.94829
[1000]	valid_0's auc: 0.94848
[1100]	valid_0's auc: 0.948633
[1200]	valid_0's auc: 0.948811
[1300]	valid_0's auc: 0.948958
[1400]	valid_0's auc: 0.949113
[1500]	valid_0's auc: 0.949173
[1600]	valid_0's auc: 0.94927
[1700]	valid_0's auc: 0.949332
[1800]	valid_0's auc: 0.949422
[1900]	valid_0's auc: 0.949477
[2000]	valid_0's auc: 0.949554
[2100]	valid_0's auc: 0.949568
[2200]	valid_0's auc: 0.949617
[2300]	valid_0's auc: 0.949636
[2400]	valid_0's auc: 0.94968
[2500]	valid_0's auc: 0.949675
[2600]	valid_0's auc: 0.949673
[2700]	valid_0's auc: 0.949654
[2800]	valid_0's auc: 0.949668
[2900]	valid_0's auc: 0.949676
[3000]	valid_0's auc: 

In [57]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective':'binary',
    'metric': {'auc'},
    'num_leaves': 1500,
    'learning_rate': 0.025,
    'feature_fraction': 0.65,
    'bagging_fraction': 0.65,
    'bagging_freq': 1,
    'verbose': 1,
    'min_data_in_leaf' : 5,
    'max_bin' : 255,
    'lambda_l1' : 0.00002,
    'lambda_l2' : 0.00001,
    'min_gain_to_split' : 0.001
}
import random
random.seed(2)
dev_users = random.sample(list(set(train1.user_id)), 55000)
dev = train1[train1.user_id.isin(dev_users)]
val = train1[train1.user_id.isin(dev_users) == False]

print(dev.shape, val.shape)
lgb_dev = lgb.Dataset(dev[indep_vars], dev['target'])
lgb_val = lgb.Dataset(val[indep_vars], val['target'])

model = lgb.train(params, lgb_dev, num_boost_round = 3000, valid_sets = (lgb_val), early_stopping_rounds = 500,
                     verbose_eval = 100)

(8973294, 93) (2384798, 93)
Training until validation scores don't improve for 500 rounds
[100]	valid_0's auc: 0.937994
[200]	valid_0's auc: 0.941056
[300]	valid_0's auc: 0.942639
[400]	valid_0's auc: 0.943368
[500]	valid_0's auc: 0.944005
[600]	valid_0's auc: 0.944409
[700]	valid_0's auc: 0.944877
[800]	valid_0's auc: 0.944934
[900]	valid_0's auc: 0.945274
[1000]	valid_0's auc: 0.945527
[1100]	valid_0's auc: 0.9457
[1200]	valid_0's auc: 0.945846
[1300]	valid_0's auc: 0.946044
[1400]	valid_0's auc: 0.946174
[1500]	valid_0's auc: 0.946257
[1600]	valid_0's auc: 0.94633
[1700]	valid_0's auc: 0.946435
[1800]	valid_0's auc: 0.946489
[1900]	valid_0's auc: 0.946566
[2000]	valid_0's auc: 0.946648
[2100]	valid_0's auc: 0.946748
[2200]	valid_0's auc: 0.946765
[2300]	valid_0's auc: 0.946851
[2400]	valid_0's auc: 0.946847
[2500]	valid_0's auc: 0.94689
[2600]	valid_0's auc: 0.946914
[2700]	valid_0's auc: 0.946985
[2800]	valid_0's auc: 0.946991
[2900]	valid_0's auc: 0.94703
[3000]	valid_0's auc: 0.9

In [61]:
pd.set_option('max_rows', 100)
pd.DataFrame({"Feature" : model.feature_name(), "Importance": model.feature_importance(importance_type='gain')}).sort_values(
    by = "Importance", ascending = False).reset_index(drop = True)

,Feature,Importance
0,n2v2_user_author_similarity,1.308349e+07
1,n2v_similarity,1.240790e+07
2,n2v1_challenge_similarity_6,8.095298e+06
3,challenge_id_diff_1,7.871943e+06
4,n2v2_challenge_similarity_2,6.109830e+06
5,challenge_id_diff_3,6.061261e+06
6,author_ID_avg_seq,5.759553e+06
7,total_submissions,5.621665e+06
8,n2v2_challenge_similarity_3,5.028645e+06
9,n2v1_challenge_similarity_9,4.435061e+06


In [62]:
pred = model.predict(val[indep_vars])
val['pred'] = pred

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
val1 = val.sort_values(by = 'pred', ascending = False).groupby('user_id').head(3)

In [64]:
val1['rank'] = val1.groupby('user_id').pred.rank(ascending = False, method = 'first')
val1 = val1.sort_values(by = ['user_id', 'pred'], ascending = False)
val1

,user_id,challenge,n2v_similarity,challenge_11,challenge_12,challenge_13,target,programming_language,challenge_series_ID,total_submissions,...,challenge_id_diff_3,challenge_id_diff_4,challenge_id_diff_5,challenge_id_diff_6,challenge_id_diff_7,challenge_id_diff_8,challenge_id_diff_9,challenge_id_diff_10,pred,rank
11529736,113837,CI25674,0.835756,CI24892,CI23990,CI25674,1,1,SI2599,1060.0,...,-378,808,625,-3,2,798,1535,1682,0.081976,1.0
11529641,113837,CI23990,0.681669,CI24892,CI23990,CI25674,1,1,SI2457,241.0,...,-2062,-876,-1059,-1687,-1682,-886,-149,-2,0.029221,2.0
11529846,113837,CI24953,0.509091,CI24892,CI23990,CI25674,0,1,SI2545,3341.0,...,-1099,87,-96,-724,-719,77,814,961,0.013620,3.0
11529117,113833,CI24436,0.846620,CI24436,CI24438,CI24445,1,1,SI2499,117.0,...,-862,-440,-1620,1,-638,-85,-10,-99,0.793916,1.0
11529053,113833,CI24440,0.746942,CI24436,CI24438,CI24445,0,1,SI2499,285.0,...,-858,-436,-1616,5,-634,-81,-6,-95,0.318366,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,4593,CI26167,0.782284,CI26166,CI26167,CI26168,1,1,SI2634,773.0,...,9,8,7,6,5,3,2,4,0.250492,2.0
1041,4593,CI26210,0.792355,CI26166,CI26167,CI26168,0,1,SI2634,1642.0,...,52,51,50,49,48,46,45,47,0.168379,3.0
786,4591,CI23691,0.710761,CI24958,CI24261,CI23648,0,1,SI2469,4823.0,...,-839,28,-23,-843,-1224,-1226,-1444,-836,0.308862,1.0
796,4591,CI24958,0.746207,CI24958,CI24261,CI23648,1,1,SI2545,6842.0,...,428,1295,1244,424,43,41,-177,431,0.126665,2.0


In [65]:
val1['precision'] = ((val1['rank'] == 1) & (val1.challenge_11 == val1.challenge)) | ((val1['rank'] ==  2) & 
                                    (val1.challenge_12 == val1.challenge)) | ((val1['rank'] == 3) & (val1.challenge_13 == val1.challenge))
val1['precision'] = val1[['user_id', 'precision']].groupby('user_id').cumsum()
val1['precision'] = val1['precision'] / val1['rank']
val1[['user_id', 'precision']].groupby('user_id').mean().precision.mean()

0.16887634951219233

In [66]:
val1['precision'] = ((val1['rank'] == 1) & (val1.target == 1)) | ((val1['rank'] ==  2) & 
                                    (val1.target == 1)) | ((val1['rank'] == 3) & (val1.target == 1))
val1['precision'] = val1[['user_id', 'precision']].groupby('user_id').cumsum()
val1['precision'] = val1['precision'] / val1['rank']
val1[['user_id', 'precision']].groupby('user_id').mean().precision.mean()

0.3771599840964055

In [67]:
## theoretical maximum

val1['rank'] = val1.groupby('user_id').target.rank(ascending = False, method = 'first')
# val1 = val1.sort_values(by = ['user_id', 'pred'], ascending = False)

val1['precision'] = ((val1['rank'] == 1) & (val1.target == 1)) | ((val1['rank'] ==  2) & 
                                    (val1.target == 1)) | ((val1['rank'] == 3) & (val1.target == 1))
val1['precision'] = val1[['user_id', 'precision']].groupby('user_id').cumsum()
val1['precision'] = val1['precision'] / val1['rank']
val1[['user_id', 'precision']].groupby('user_id').mean().precision.mean()

0.4233759977979519

In [68]:
## theoretical maximum
val1 = val.sort_values(by = 'target', ascending = False).groupby('user_id').head(3)
val1['rank'] = val1.groupby('user_id').target.rank(ascending = False, method = 'first')
# val1 = val1.sort_values(by = ['user_id', 'pred'], ascending = False)

val1['precision'] = ((val1['rank'] == 1) & (val1.target == 1)) | ((val1['rank'] ==  2) & 
                                    (val1.target == 1)) | ((val1['rank'] == 3) & (val1.target == 1))
val1['precision'] = val1[['user_id', 'precision']].groupby('user_id').cumsum()
val1['precision'] = val1['precision'] / val1['rank']
val1[['user_id', 'precision']].groupby('user_id').mean().precision.mean()

0.8754358197999633

In [69]:
del dev
del val
gc.collect()

36

In [70]:
%%time
lgb_train = lgb.Dataset(train1[indep_vars], train1['target'])
model = lgb.train(params, lgb_train, num_boost_round = 3000)
pred = model.predict(test1[indep_vars])
test1['pred'] = pred

CPU times: user 16h 58min 52s, sys: 2h 23min 22s, total: 19h 22min 15s
Wall time: 1h 15min 43s


In [71]:
test2 = test1.sort_values(by = 'pred', ascending = False).groupby('user_id').head(3)

In [72]:
test2['rank'] = test2.groupby('user_id').pred.rank(ascending = False, method = 'first')
test2 = test2.sort_values(by = ['user_id', 'pred'], ascending = False)
test2['user_sequence'] = test2.user_id.astype(str) + "_1" + test2['rank'].astype(int).astype(str)
test2[['user_sequence', 'challenge']].to_csv("./submission.csv", index = False)